<a href="https://colab.research.google.com/github/YASsMeN1997/Music-Generation/blob/main/App_Music_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [1]:
### note: uncomment this part when you run the code ###

!apt-get update
!pip install streamlit==1.13.0
!pip install pyngrok
!apt install unzip
!pip install streamlit_lottie
!pip install streamlit-option-menu
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok
!pip install hydralit_components
!pip install branca==0.3.1
!pip install streamlit==1.13.0
!pip install midi2audio
!pip install pretty_midi
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu b

In [2]:
import warnings
warnings.filterwarnings("ignore")



# GOOGLE DRIVE

In [3]:

# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# streamlit app file


- run the below cell to write your app.py file

In [4]:
%%writefile app.py
import streamlit as st
import numpy as np
from streamlit_lottie import st_lottie
from PIL import Image
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import matplotlib as mpl
import requests
import streamlit as st
import streamlit.components.v1 as components
import hydralit_components as hc
import time
import glob
import pickle
import numpy
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import IPython
from IPython.display import Image, Audio
from music21 import *
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau , EarlyStopping
from music21 import instrument, note, stream, chord
import io
import numpy as np
import pretty_midi
import requests
import streamlit as st
from bs4 import BeautifulSoup
from scipy.io import wavfile
from midi2audio import FluidSynth



####################### prediction ################################

# generate music
def generate(seq_len,x):
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('/content/drive/MyDrive/Colab Notebooks/final project/model40_3/final_notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab , seq_length = seq_len)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab, x)
    create_midi(prediction_output)


def prepare_sequences(notes, pitchnames, n_vocab , seq_length):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = seq_length
    network_input = []
    normalized_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    adam = tf.keras.optimizers.Adam(0.001)

    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(256))
    model.add(BatchNorm())
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.2))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
     # 'rmsprop'
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    # Load the weights to each node
    model.load_weights('/content/drive/MyDrive/Colab Notebooks/final project/model40_3/best2.h5')

    return model


def generate_notes(model, network_input, pitchnames, n_vocab , x):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate x notes (x entered by user)
    for note_index in range(x):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output


def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            
        # pattern is a rest
        elif pattern == 'r':
            new_note = note.Rest(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
            
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output2.mid')




###########################################################################







    
####################### stream lit app ################################


st.set_page_config(page_title="Music Generation", page_icon=":tada:", layout="wide")
def load_lottieurl(url):
    r = requests.get(url)
    if r.status_code != 200:
        return None
    return r.json()

# ---- LOAD ASSETS ----
lottie_coding = load_lottieurl("https://assets5.lottiefiles.com/private_files/lf30_fjln45y5.json")


# ---- HEADER SECTION ----
with st.container():
    left_column, right_column = st.columns(2)
    with left_column:
        st.subheader("Music Generation :musical_keyboard:")
        st.write(
            "Our website is an application of piano music generation, you can listen to new musical notes generated by LSTM artificial neural network, which is used in fields of AI and deep learning. Let's get it started :notes:"
        )
    with right_column:
        st_lottie(lottie_coding, height=300, key="coding")


with st.sidebar:
  len_notes = st.slider('Please Choose The Notes Length', 20, 750, 20, 4)
  st.write("Notes Length = " , len_notes)

if st.sidebar.button('Generate My Music'):
  if len_notes:

    with st.container():
        st.write("---")

        with hc.HyLoader('✨ Your music is now under processing ✨' ,hc.Loaders.standard_loaders,index=[3,0,5]):
          time.sleep(10)
          generate(10,len_notes)

          fs = FluidSynth('font.sf2' ,sample_rate=44100 )

          fs.midi_to_audio('/drive/ngrok-ssh/test_output2.mid', 'output.wav')

          st.audio('output.wav')
          st.markdown("Here you are! You can download your music by right-clicking on the media player")







Writing app.py


# run streamlit app

In [5]:
!ls

app.py	font.sf2  ngrok  ngrok-stable-linux-amd64.zip


In [6]:
# run this cell with your authtoken from ngrok website after create an account on it (https://ngrok.com/)
# then run the next cell and click on the output link of this cell to open your streamlit app and 

from pyngrok import ngrok 
!ngrok authtoken "2FpXLfPnzvs4s2S9knmtVCnmWHX_5kFivZ7E1NjyGvnBQozpj"
!nohup streamlit run app.py & 

url = ngrok.connect(port = 8501)
url

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
nohup: appending output to 'nohup.out'


<NgrokTunnel: "http://d90a-34-68-99-109.ngrok.io" -> "http://localhost:80">

In [7]:
!streamlit run --server.port 80 app.py >/dev/null


2022-11-20 17:43:16.110 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
2022-11-20 17:46:03.955642: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-11-20 17:46:03.989 Layer lstm will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.
2022-11-20 17:46:04.300 Layer lstm_1 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.
2022-11-20 17:50:39.017 Layer lstm will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.
2022-11-20 17:50:39.263 Layer lstm_1 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


In [8]:
# this cell to kill the process or stop the app

ngrok.kill()

